In [1]:
# module
import numpy as np
import pandas as pd
import tqdm
from haversine import haversine
from tqdm import tqdm_pandas, tqdm

In [2]:
#load data
cw = pd.read_csv('횡단보도_완료.csv', encoding = 'cp949')
fluid = pd.read_csv('버스좌표및승하차인원.csv', encoding = 'cp949')

In [3]:
fluid = fluid[['정류소명', 'Y좌표', 'X좌표', 'on_bus', 'off_bus']]

In [4]:
# 차 사고
cw.columns = ['crosswalk_id', 'cw_lon', 'cw_lat']

# 횡단보도
fluid.columns = ['bus', 'sub_lat', 'sub_lon', 'on_bus', 'off_bus']

In [5]:
# key 만들기
cw['key'] = 0
fluid['key'] = 0

In [6]:
# 데이터 합치기
prod = pd.merge(cw, fluid, on = 'key', how = 'outer')

In [7]:
## 하버사인 거리
lat1 = np.deg2rad(prod['cw_lat'])
lat2 = np.deg2rad(prod['sub_lat'])
lon1 = np.deg2rad(prod['cw_lon'])
lon2 = np.deg2rad(prod['sub_lon'])
dlat = lat2 - lat1 
dlon = lon2 - lon1 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [8]:
# 미터거리 구하기
prod['distance'] = c * 6371 * 1000

In [9]:
# 상세한 확인
prod['distance'].describe()

count    2.600242e+07
mean     1.494649e+04
std      7.023483e+03
min      1.131303e+00
25%      9.357713e+03
50%      1.491395e+04
75%      2.049284e+04
max      3.400875e+04
Name: distance, dtype: float64

In [10]:
# 거리가 가까운 순서대로 정렬
p = prod.sort_values(by=['distance'])

In [11]:
# 가장 가까운 값들로 저장
pp = p.drop_duplicates(subset = 'crosswalk_id')[['crosswalk_id', 'bus', 'on_bus', 'off_bus', 'distance']]

In [12]:
pp.to_csv('버스_유동인구_거리포함.csv', index = False, encoding = 'cp949')

In [13]:
pp['distance'].describe()

count    2442.000000
mean       99.424461
std        80.819625
min         1.131303
25%        42.309514
50%        81.846217
75%       136.187891
max      1040.265502
Name: distance, dtype: float64

In [18]:
pp['bus'].nunique()

340